In [ ]:
import os
import cv2
import base64
from mistralai import Mistral

MISTRAL_API_KEY =  "***REMOVED***"
model = "pixtral-12b-2409"
output_folder = 'extracted_frames'

client = Mistral(api_key=MISTRAL_API_KEY)
model = "pixtral-12b-2409"

def extract_frames(video_path, output_folder, interval=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_frames = []

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        if frame_count % interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
            extracted_frames.append(frame_filename)

        frame_count += 1

    video_capture.release()
    return extracted_frames

def encode_image_to_base64(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')
    
def query_model(video_path, prompt): 
    frames = extract_frames(video_path, output_folder)
    contents = [{
        "type": "text",
        "text": prompt,
    }]

    for frame_path in frames:
        encoded_image = encode_image_to_base64(frame_path)
        contents.append({
            "type": "image_url",
            "image_url": f"data:image/jpeg;base64,{encoded_image}"
        })

    messages = [
        {
            "role": "user",
            "content": contents
        }
    ]
    
    response = client.chat.complete(model=model, messages=messages)

    return response.choices[0].message.content
    

video_path = 'tesla-real-world-video-q-a/videos/videos/00001.mp4'
prompt = '''
Was ego doing a legal maneuver if its goal is to turn right at the intersection? 

Options:

(A) It's legal as the lane is empty. (B) It's illegal as the right turn lane is bloacked by construction. (C) It's illegal as ego was cutting in other vehicles that were waiting. (D) It's legal but the lane ahead is way too narrow for ego to pass.
'''

response = query_model(video_path, prompt)
response


"Based on the image and the options provided, the correct answer is:\n\n(B) It's illegal as the right turn lane is blocked by construction.\n\nThe right turn lane is visibly blocked by construction barriers and orange cones, making it illegal for the ego vehicle to turn right at the intersection."